In [1]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [2]:
import logging
import pandas as pd

from general_functions.return_account_ids import return_account_ids
from general_functions.constants import return_api_url
from general_functions.call_api_with_account_id import call_api_with_accountId, send_to_innkeepr_api_paginated

In [3]:
url = return_api_url()
print(f"url = {url}")
accounts = return_account_ids()

In [4]:
all_conversion_actions = pd.DataFrame()
for account in accounts:
    account_id = account["id"]
    customer = account["name"]
    print(f"customer = {customer}")
    conversion_actions = call_api_with_accountId(
        f"{url}/sources/googleAdwords/conversionActions/query",
        account_id,
        {},
        logging,
    )
    if len(conversion_actions) == 0:
        continue
    conversion_actions = pd.json_normalize(conversion_actions)
    conversion_actions["account"] = customer
    all_conversion_actions = pd.concat([all_conversion_actions, conversion_actions])

In [5]:
all_conversion_actions.sort_values(by="created").reset_index(drop=True)

In [6]:
temp = all_conversion_actions.sort_values(by=["account","created"])[["id","adAccountId","account","name","goalId","type","created"]]
temp.to_csv("conversion_actions.csv", index=False)

In [7]:
temp

In [8]:
all_conversion_actions[["account","id","name"]].rename(columns={"id":"conversionActionId"}).to_dict(orient="records")